In [34]:
using Random
using ProgressBars
using DataFrames
using Tables
using CSV
using ProgressMeter

In [16]:
"""Returns the nearest neighbor sum of (i,j) in the matrix x, obeying periodic boundary conditions"""
function nn(x::AbstractMatrix, i::Integer, j::Integer)
    x[mod1(i-1, size(x, 1)), j] + x[mod1(i+1, size(x, 1)), j] + x[i, mod1(j-1, size(x, 2))] + x[i, mod1(j+1, size(x, 2))]
end

"""Perform one step in the Metropolis algorithm for the 2D Ising model stored in x,
with the net magnetization and energy E."""
function move!(x::AbstractMatrix, lookup::AbstractVector)
    #= Pick a random site =#
    i = rand(1:size(x, 1))
    j = rand(1:size(x, 2))

    #= Compute the Metropolis acceptance probability =#
    nn_sum = nn(x, i, j) # to avoid multiple function calls

    # there is only 5 possible outcomes with energies -4, -2, 0, 2, 4
    index = trunc(Int64, (nn_sum+6)÷2)

    p_acc = (x[i, j] == 1) ? lookup[6-index] : lookup[index]

    #= Perform the single spin flip =#
    if p_acc > rand()
        x[i, j] *= -1
    end
    
    x
end

move!

In [17]:
function thermalize_grid(beta,L,Nthermalization)
    lookup = exp.(2*beta*[-4:2:4...]) # lookup table for the allowed Metropolis
    
    #= Initialization of the ferromagnetic ground state =#
    x = ones(L, L)       # all spins up
    
    #= Thermalization =#
    for _ in 1:Nthermalization
        x = move!(x, lookup)
    end
    x
end

thermalize_grid (generic function with 1 method)

## Create the training set

In [36]:
temp = LinRange(0.2,0.8,20)
L = 30
Nthermalization = 10e5
labels = []
inputs = []

@showprogress 1 "Computing..."  for beta ∈ temp
    print("running for beta = ", beta)
    for i in 1:1000
        x = reshape(thermalize_grid(beta,L,Nthermalization),(1,:))
        push!(labels,beta)
        inputs = cat(inputs,x,dims=1)
    end
end

df = DataFrame(inputs, :auto)
df.label = labels
CSV.write("training_set.csv", df)

Computing... 100%|███████████████████████████████████████| Time: 0:25:15


running for beta = 0.2running for beta = 0.23157894736842108running for beta = 0.2631578947368421running for beta = 0.2947368421052632running for beta = 0.3263157894736842running for beta = 0.35789473684210527running for beta = 0.3894736842105263running for beta = 0.42105263157894735running for beta = 0.45263157894736844running for beta = 0.48421052631578954running for beta = 0.5157894736842106running for beta = 0.5473684210526316running for beta = 0.5789473684210527running for beta = 0.6105263157894737running for beta = 0.6421052631578947running for beta = 0.6736842105263159running for beta = 0.7052631578947368running for beta = 0.7368421052631579running for beta = 0.768421052631579running for beta = 0.8

"training_set.csv"

## Create the testing set

In [38]:
temp = LinRange(0.2,0.8,25)
L = 30
Nthermalization = 10e5
labels = []
inputs = []

@showprogress 1 "Computing..." for beta ∈ temp
    print("running for beta = ", beta)
    for i in 1:1000
        x = reshape(thermalize_grid(beta,L,Nthermalization),(1,:))
        push!(labels,beta)
        inputs = cat(inputs,x,dims=1)
    end
end

df = DataFrame(inputs, :auto)
df.label = labels
CSV.write("validation_set.csv", df)

Computing... 100%|███████████████████████████████████████| Time: 0:25:24


running for beta = 0.2running for beta = 0.225running for beta = 0.25running for beta = 0.275running for beta = 0.30000000000000004running for beta = 0.325running for beta = 0.35000000000000003running for beta = 0.375running for beta = 0.4running for beta = 0.42500000000000004running for beta = 0.45running for beta = 0.47500000000000003running for beta = 0.5running for beta = 0.525running for beta = 0.55running for beta = 0.575running for beta = 0.6running for beta = 0.6250000000000001running for beta = 0.6500000000000001running for beta = 0.6749999999999999running for beta = 0.7000000000000001running for beta = 0.7250000000000001running for beta = 0.7500000000000001running for beta = 0.775running for beta = 0.8

"validation_set.csv"

In [1]:
temp = LinRange(0.2,0.8,20)

20-element LinRange{Float64, Int64}:
 0.2,0.231579,0.263158,0.294737,0.326316,…,0.705263,0.736842,0.768421,0.8